<a href="https://colab.research.google.com/github/MStamirski/Faces/blob/main/Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageFilter
import random
from keras.metrics import Precision
from keras.metrics import Recall

In [ ]:
def prepare_data(input, output, char_for_positive):

  def get_target_values(target_chars, lab_char):
    targets = [1.0 if x==lab_char else 0.0 for x in target_chars]
    return np.array(targets)

  X = input / 255
  y = get_target_values(output, char_for_positive)
  return X, y

In [ ]:
def plot_history(history):
  hist = history.history
  
  fig, ax = plt.subplots(figsize=[8,6])
  ax.grid()
  ax.plot(hist['loss'], 'r',linewidth=3.0, label='training')
  ax.plot(hist['val_loss'], 'b',linewidth=3.0, label='validation')
  ax.set_xlabel('Epochs')
  ax.legend()
  ax.set_title('Loss function')
  plt.show()

  fig, ax = plt.subplots(figsize=[8,6])
  ax.grid()
  ax.plot(hist['precision'], 'r',linewidth=3.0, label='training')
  ax.plot(hist['val_precision'], 'b',linewidth=3.0, label='validation')
  ax.set_xlabel('Epochs')
  ax.legend()
  ax.set_title('Precision')
  plt.show()

  fig, ax = plt.subplots(figsize=[8,6])
  ax.grid()
  ax.plot(hist['recall'], 'r',linewidth=3.0, label='training')
  ax.plot(hist['val_recall'], 'b',linewidth=3.0, label='validation')
  ax.set_xlabel('Epochs')
  ax.legend()
  ax.set_title('Recall')
  plt.show()

In [ ]:
def show_metrics(y_test, y_pred):
  precision = Precision()
  precision.update_state(y_test, y_pred)
  prec_res = precision.result()
  print(f"Precision: {prec_res}")

  recall = Recall()
  recall.update_state(y_test, y_pred)
  rec_res = recall.result()
  print(f"Recall: {rec_res}")

  F1 = 2 * (prec_res * rec_res) / (prec_res + rec_res)
  print(f"F1 Score: {F1}")

In [ ]:
def get_best_threshold(y_test, y_pred):

  thresholds = list(np.arange(0.05, 1.0, 0.05))

  prec = Precision(thresholds = thresholds)
  prec.update_state(y_test, y_pred)
  precisions = prec.result()

  rec = Recall(thresholds = thresholds)
  rec.update_state(y_test, y_pred)
  recalls = rec.result()

  max_F1 = 0
  best_threshold = 0.5
  for precision, recall, threshold in zip(precisions, recalls, thresholds):
    F1 = 2 * (precision * recall) / (precision + recall)
    if F1 > max_F1:
      max_F1 = F1
      best_threshold = threshold

  return best_threshold

In [ ]:
def get_tuned_prediction(y_pred, threshold):
  return np.array([[1.0 if lab>= threshold else 0.0 for lab in labs] for labs in y_pred])

In [ ]:
def make_prediction(model, X_test, y_test):
  y_pred = model.predict(X_test)
  print("\nMetrics before tuning")
  show_metrics(y_test, y_pred)

  threshold = get_best_threshold(y_test, y_pred)
  print(f"\nBest threshold: {threshold}")

  y_pred_tuned = get_tuned_prediction(y_pred, threshold)
  print("\nMetrics after tuning")
  show_metrics(y_test, y_pred_tuned)

  return y_pred_tuned

In [ ]:
def show_prediction(ind, faces, targets, targets_pred):
  face = Image.fromarray(faces[ind]*255)  # faces were normalized
  face.show()
  print(f"Label value: {int(targets[ind])}, Prediction: {int(targets_pred[ind][0])}\n")

In [ ]:
def show_predictions(faces, targets, targets_pred):

  cls0_ind_list  = [ind for (ind, value) in enumerate(list(targets)) if round(value,0) == 0]
  cls1_ind_list  = [ind for (ind, value) in enumerate(list(targets)) if round(value,0) == 1]

  cls0_ind_shortlist = random.choices(cls0_ind_list, k=5)
  cls1_ind_shortlist = random.choices(cls1_ind_list, k=5)

  print("============ Predictions for label value 0 ===================")
  for ind in cls0_ind_shortlist:
    show_prediction(ind, faces, targets, targets_pred)
  
  print("\n============ Predictions for label value 1 ===================")
  for ind in cls1_ind_shortlist:
    show_prediction(ind, faces, targets, targets_pred)